In [ ]:
import numpy as np

Parser to read in matrix in phylip format

In [1]:
def read_phylip(file):
    n = None
    scoring_matrix = []
    taxons = []

    with open(file, "r") as f:
        lines = f.readlines()

        for line in lines:
            if line == lines[0]:
                n = int(line.rstrip())
            else:
                current_line = line.split()
                row = []
                for char in current_line:
                    if char == current_line[0]:
                        taxons.append(char)
                    else:
                        row.append(float(char))
                scoring_matrix.append(row)
    return n, taxons, scoring_matrix

NJ algorithm uses an N matrix to quantify relatedness

In [2]:
def N_mat(dist_matrix):
    """
    Dist matrix must be a numpy array
    """
    size = dist_matrix.shape[0]
    
    # get r
    r = np.sum(dist_matrix, axis = 0)/(size-2)
    
    # calculate N matrix
    N = np.zeros((size, size))
    
    # save min coordinates while iterating
    min_val = 0
    ij = 0, 0
    
    for i in range(size):
        for j in range(size):
            if i > j:
                nij = dist_matrix[i, j] - (r[i] + r[j])
                N[i, j] = nij
                
                if nij < min_val:
                    min_val = nij
                    ij = i, j
                
    # fill the other half
    N = N.T + N
    
    return r, N, ij

The following creates biopython tree on basis of a distance matrix

In [3]:
def make_tree(file):
    
    parsed = read_phylip(file)
    
    m = np.array(parsed[2])
    taxons = parsed[1]
    nodes = dict()
    
    while m.shape[1] > 3:
        r, N, ij = N_mat(m)
        ki = 1/2*(m[ij]+r[ij[0]]-r[ij[1]])
        kj = 1/2*(m[ij]+r[ij[1]]-r[ij[0]])

        # Check if children is in dictionary
        child1 = taxons[ij[0]]
        child2 = taxons[ij[1]]

        if child1 not in nodes:
            nodes[child1] = Phylo.BaseTree.Clade(branch_length = ki, name = child1)
        if child2 not in nodes:
            nodes[child2] = Phylo.BaseTree.Clade(branch_length = kj, name = child2)

        # Add node to dictionary
        name = taxons[ij[0]] + "_" + taxons[ij[1]]
        nodes[name] = Phylo.BaseTree.Clade(branch_length = None, name = name, clades = [nodes[child1], nodes[child2]])

        # Pop names from taxons list
        taxons.pop(ij[0])
        taxons.pop(ij[1])
        taxons.append(name)

        # Add k to dist matrix
        k = list(1/2 * (m[ij[0],:]+m[ij[1],:] - m[ij]))
        k.pop(ij[0])
        k.pop(ij[1])
        k.append(0)

        # Modify distance matrix
        k_col = np.reshape(k[0:-1], (-1, 1))
        k_row = np.reshape(k, (1, -1))
        m = np.delete(m, ij, 1)
        m = np.delete(m, ij, 0)
        m = np.append(m, k_col, axis = 1)
        m = np.append(m, k_row, axis = 0)
    
    final_dist = {taxons[0] : (m[0,1]+m[0,2]-m[1,2])/2,
              taxons[1] : (m[0,1]+m[1,2]-m[0,2])/2,
              taxons[2] : (m[0,2]+m[1,2]-m[0,1])/2}
    
    for child in taxons:
        if child not in nodes:
            nodes[child] = Phylo.BaseTree.Clade(branch_length = final_dist[child], name = child)
        else:
            nodes[child] = Phylo.BaseTree.Clade(branch_length = final_dist[child], 
                                                name = child, 
                                                clades = nodes[child].clades)
    
    name = "_".join(taxons)
 
    nodes[name] = Phylo.BaseTree.Clade(branch_length = None, 
                                                name = name, 
                                                clades = [nodes[taxons[0]], nodes[taxons[1]], nodes[taxons[2]]])
    tree = Phylo.BaseTree.Tree(root = nodes[name])
    
    return tree

Example of use

In [ ]:
test = "example_slide4.phy"
t = make_nodes(test)

# P
t.format("newick")